In [1]:
import requests 
import json
import FAIR

## First get authorized

In [ ]:
r = requests.post('https://clarklab.uvarc.io/auth/logout',headers = {"Authorization":token })
r.content.decode()

Visit:
<br>
https://clarklab.uvarc.io/auth/login
<br>
And copy and paste access token below

In [21]:
token = "Agy5d87GWaVB48yjxMBKpJX2PwM7MjXo00arEKle5mYbqWVP3WUVC6gkB0Nbxm6g16aV7rB4ynOjElCOj8qmysW1dyHXwJQiN2kVSv78ohM9Wo"

In [25]:
requests.post('https://clarklab.uvarc.io/auth/inspect',headers = {'Authorization':'Agdye4XpNrdq1rpd46EM6NVe3rq6KxD998m4aXDMDVWGyOq997TXCxzooP5oNq088z6e7r42D6BgD0hgQXym5f9Wk8IkmyMFMp9KF152DunqJy'})

<Response [204]>

## Using Transfer Service upload data with metadata

Upload a file to minio via the upload_file function.
<br>
<br>
upload_file takes:
        - file_path: path to local file 
        - metadata: describes the file to be uploaded
returns:
    - PID representing object
upload_file then calls the transfer service which uploads the file to minio and metadata to mongoDB and Stardog

In [22]:
dataset_meta = {
    "@context":{
        "@vocab":"http://schema.org/"
    },
    "@type":"Dataset",
    "name":"Raw Data",
    "description":"Heart Rate Measures from patient from admission to discharge.",
    "author":[
        {
            "name":"Justin Niestroy",
            "@id": "https://orcid.org/0000-0002-1103-3882",
            "affiliation":"University of Virginia"
        }
    ],
}
data_id = FAIR.upload_file('UVA_7129_HR2.csv',dataset_meta,token = token)
data_id

'ark:99999/75120e66-59f1-4346-a7c2-caeb69a5011e'

## Check MDS to make sure metadata was uploaded correctly

retrieve_metadata calls MDS to get the metadata for the newly minted id.
<br>
<br>
retrieve_metadata takes:
    - PID: identifier of interest
returns:
    - json-ld metadata of given PID

In [6]:
FAIR.retrieve_metadata(data_id,token = token)

{'@context': {'@vocab': 'http://schema.org/'},
 '@id': 'ark:99999/8e93e470-608f-4f26-bd51-651d7632cbf9',
 '@type': 'Dataset',
 'author': [{'@id': 'https://orcid.org/0000-0002-1103-3882',
   'affiliation': 'University of Virginia',
   'name': 'Justin Niestroy'}],
 'description': 'Heart Rate Measures from patient from admission to discharge.',
 'distribution': [{'@id': 'ark:99999/1a5a4b08-f852-472b-b189-823471a3bf6c',
   '@type': 'DataDownload',
   'contentSize': 201072,
   'contentUrl': 'minio:9000/breakfast/UVA_7129_HR2.csv',
   'fileFormat': 'csv',
   'name': 'UVA_7129_HR2.csv'}],
 'name': 'Raw Data',
 'sdPublicationDate': '2020-08-11T14:04:42.205709643Z',
 'url': 'http://ors.uvadcos.io/ark:99999/8e93e470-608f-4f26-bd51-651d7632cbf9'}

## Upload Spark Script to run on newly uploaded data

In [7]:
spark_code_meta = {
    "@context":{
        "@vocab":"http://schema.org/"
    },
    "@type":"SoftwareSourceCode",
    "name":"Processing  Script",
    "description":"Sample Source Code for HCTSA test on spark",
    "author":[
        {
            "name":"Justin Niestroy",
            "@id": "https://orcid.org/0000-0002-1103-3882",
            "affiliation":"University of Virginia"
      }
    ]
}
software_id = FAIR.upload_file('run_algos.py',spark_code_meta,token = token)
software_id

'ark:99999/067fbda2-0cdb-4687-9ed1-3eeeb8ff9d61'

## Submit Spark Job Using Compute Service

Run a the uploaded script on the uploaded data by calling compute
<br>
<br>
FAIR.compute takes: 
    - data pid 
    - software pid
    - type of job ie spark,nipype,custom  
compute then launches kubernetes pods to run the given script on the provided data using specialized pods
<br>
<br>
compute returns:
    - job_PID 

In [8]:
job_id = FAIR.compute(data_id,software_id,'spark',token = token)
job_id

'ark:99999/3623aaed-b144-487f-be16-5dca419f5c69'

## Watch Running Job
list_running_jobs takes:
<br>
<br>
list_running_jobs returns:
    - list of running job PIDs

In [10]:
FAIR.list_running_jobs(token = token)

[]

## Get Job outputs from job metadata

Upon completion of a job the compute service updates the job id metadata to include properties:
    - evi:supports: list of output PIDs 
    - logs: logs from job

In [11]:
job_metadata = FAIR.retrieve_metadata(job_id,token = token)
output_ids = job_metadata.get('evi:supports')
output_ids

[{'@id': 'ark:99999/77e79e97-98e1-4d34-a11b-dc6eb1e7561a'},
 {'@id': 'ark:99999/48b8d22d-4859-48cc-8d2b-e4b4312d3b03'}]

The computation created 2 outputs
<br>
Metadata for the two created outputs is shown below

In [12]:
output_metadata = [ 
   FAIR.retrieve_metadata(out['@id'],token = token) for out in output_ids
]
output_metadata

[{'@context': {'@vocab': 'http://schema.org/'},
  '@id': 'ark:99999/77e79e97-98e1-4d34-a11b-dc6eb1e7561a',
  '@type': 'Dataset',
  'distribution': [{'@id': 'ark:99999/3d196a77-a3b7-4f83-ba2c-b901bd69eef5',
    '@type': 'DataDownload',
    'contentUrl': 'minio:9000/breakfast/3623aaed-b144-487f-be16-5dca419f5c69/_SUCCESS',
    'fileFormat': '_SUCCESS',
    'name': '_SUCCESS'}],
  'evi:generatedBy': {'@id': 'ark:99999/3623aaed-b144-487f-be16-5dca419f5c69'},
  'name': '_SUCCESS',
  'sdPublicationDate': '2020-08-11T14:07:04.02756933Z',
  'url': 'http://ors.uvadcos.io/ark:99999/77e79e97-98e1-4d34-a11b-dc6eb1e7561a'},
 {'@context': {'@vocab': 'http://schema.org/'},
  '@id': 'ark:99999/48b8d22d-4859-48cc-8d2b-e4b4312d3b03',
  '@type': 'Dataset',
  'distribution': [{'@id': 'ark:99999/4bf78bfe-251f-41e2-963d-4aa3ef37a6ac',
    '@type': 'DataDownload',
    'contentUrl': 'minio:9000/breakfast/3623aaed-b144-487f-be16-5dca419f5c69/part-00000-140fd052-992e-4386-8552-108f89cfe961-c000.csv',
    'fileF

## Upload Image Code

In [13]:
spark_graphic_meta = {
    "@context":{
        "@vocab":"http://schema.org/"
    },
    "@type":"SoftwareSourceCode",
    "name":"Image Script",
    "description":"Creates heatmap of patients stay",
    "author":[
        {
            "name":"Justin Niestroy",
            "@id": "https://orcid.org/0000-0002-1103-3882",
            "affiliation":"University of Virginia"
      }
    ]
}
graphic_code_id = FAIR.upload_file('make_graphic.py',spark_graphic_meta,token = token)

## Run Spark Job to create Image

In [14]:
image_job_id = FAIR.compute(output_ids[1]['@id'],graphic_code_id,'spark',token = token)
image_job_id

'ark:99999/077d0282-1318-4fb2-84b9-52b6620cd655'

## Check on running jobs

In [16]:
FAIR.list_running_jobs(token = token)

[]

### Get image ID from Updated Job ID

In [17]:
image_job_meta = FAIR.retrieve_metadata(image_job_id,token = token)
image_id = image_job_meta['evi:supports'][0]['@id']

## Get the image metadata

In [18]:
FAIR.retrieve_metadata(image_id,token = token)

{'@context': {'@vocab': 'http://schema.org/'},
 '@id': 'ark:99999/56e22bf2-8835-46e2-b80d-3225967ee429',
 '@type': 'Dataset',
 'distribution': [{'@id': 'ark:99999/817433b7-3db2-42b4-bc33-e31fb2473e38',
   '@type': 'DataDownload',
   'contentUrl': 'minio:9000/breakfast/077d0282-1318-4fb2-84b9-52b6620cd655/Histogram_Heatmap.png',
   'fileFormat': 'png',
   'name': 'Histogram_Heatmap.png'}],
 'evi:generatedBy': {'@id': 'ark:99999/077d0282-1318-4fb2-84b9-52b6620cd655'},
 'name': 'Histogram_Heatmap.png',
 'sdPublicationDate': '2020-08-11T14:10:11.284196878Z',
 'url': 'http://ors.uvadcos.io/ark:99999/56e22bf2-8835-46e2-b80d-3225967ee429'}

### Build Evidence graph of created image using the evidence graph service

Evidence Graph Service builds an json-ld evidence graph representing the provenance (datasets, software, and computations that support the PID)
<br>
<br>
evidence_graph takes:
    - PID
evidence_graph returns:
    - json-ld evidence graph for given PID

In [19]:
FAIR.evidence_graph(image_id,token = token)

{'@context': {'@vocab': 'http://schema.org/', 'evi': 'http://w3id.org/EVI#'},
 '@id': 'ark:99999/56e22bf2-8835-46e2-b80d-3225967ee429',
 '@type': 'Dataset',
 'evi:generatedBy': {'@id': 'ark:99999/077d0282-1318-4fb2-84b9-52b6620cd655',
  '@type': 'evi:Computation',
  'evi:usedDataset': {'@id': 'ark:99999/48b8d22d-4859-48cc-8d2b-e4b4312d3b03',
   '@type': 'Dataset',
   'evi:generatedBy': {'@id': 'ark:99999/3623aaed-b144-487f-be16-5dca419f5c69',
    '@type': 'evi:Computation',
    'evi:usedDataset': {'@id': 'ark:99999/8e93e470-608f-4f26-bd51-651d7632cbf9',
     '@type': 'Dataset',
     'author': {'@id': 'https://orcid.org/0000-0002-1103-3882',
      '@type': 'Person',
      'name': 'Justin Niestroy'},
     'name': 'Raw Data'},
    'evi:usedSoftware': {'@id': 'ark:99999/067fbda2-0cdb-4687-9ed1-3eeeb8ff9d61',
     '@type': 'SoftwareSourceCode',
     'author': {'@id': 'https://orcid.org/0000-0002-1103-3882',
      '@type': 'Person',
      'name': 'Justin Niestroy'},
     'name': 'Processing 

## View Visualization of the Evidence Graph

In [20]:
'https://clarklab.uvarc.io/viz/' + image_id

'https://clarklab.uvarc.io/viz/ark:99999/56e22bf2-8835-46e2-b80d-3225967ee429'

## Download Created Image

Download output files from computations using download_file
<br>
<br>
download_file takes:
    - PID: id representing object to be downloaded
    - name: file name for downloaded object
<br>
download_file returns:<br>
         - bool: True if download if succesful<br>
         - file saved location given

In [ ]:
from IPython.display import Image
FAIR.download_file(image_id,'Created Image.png',token = token)
Image(filename='Created Image.png') 